# ELT using PySpark on Local with S3

In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

การที่เราจะเชื่อมต่อไปยัง Amazon S3 เราจำเป็นต้องมี AWS Access Key ID และ AWS Secret Access Key ก่อน และถ้าเป็นการเชื่อมต่อแบบ temporary เราจะต้องใช้ AWS Session Token อีกค่าหนึ่งด้วย

การที่เราจะได้มาทั้ง 3 ค่านั้น สามารถทำได้โดยไปที่ AWS Learner Lab ของคอร์สนี้ และที่ Terminal ให้เราพิมพ์คำสั่ง `cat ~/.aws/credentials` ลงไป เราจะได้ค่าทั้ง 3 ค่าที่เราต้องการมา ตามรูปด้านล่างนี้

<img src="./img/getting-aws-keys.png" width="800" />

In [2]:
AWS_ACCESS_KEY_ID = "YOUR_AWS_ACCESS_KEY_ID"
AWS_SECRET_ACCESS_KEY = "YOUR_AWS_SECRET_ACCESS_KEY"
AWS_SESSION_TOKEN = "YOUR_AWS_SESSION_TOKEN"

In [3]:
conf = SparkConf()
conf.set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider")
conf.set("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY_ID)
conf.set("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY)
conf.set("spark.hadoop.fs.s3a.session.token", AWS_SESSION_TOKEN)

In [4]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

ก่อนที่เราจะดึงข้อมูลจาก S3 ได้ เพื่อความงานในการทำโปรเจค เราจะเซต Bucket ของเราให้เป็นแบบ public access (ซึ่งตรงนี้ไม่ recommend ให้เอาไปใช้ในงานจริง เนื่องจากจะทำให้ข้อมูลขององค์กรรั่วไหลได้) ให้เราไปที่ Bucket ที่เก็บข้อมูลของเรา ที่ section "Block public access (bucket settings)" ให้เรากด Edit แล้วเอาติ๊กถูกออกที่ Block all public access ออก ตามรูปด้านล่าง แล้วเซฟ

<img src="./img/open-public-access.png" width="500" />

หลังจากนั้นเราต้องเซต Bucket policy เพิ่มเติมเพื่อกำหนดว่าจะให้ public access สามารถทำอะไรได้บ้าง ให้ไปที่ section "Bucket policy" แล้วกด Edit แล้วใส่ JSON ด้านล่างนี้ไปที่ช่อง Policy เสร็จแล้วกดเซฟ


```json
{
    "Version": "2012-10-17",
    "Id": "Policy1670253933184",
    "Statement": [
        {
            "Sid": "Stmt1670253929820",
            "Effect": "Allow",
            "Principal": "*",
            "Action": "*",
            "Resource": "arn:aws:s3:::kanouivirach-titanic/*"
        }
    ]
}
```

ผลลัพธ์ที่ได้จะมีหน้าตาคล้ายรูปด้านล่างแบบนี้

<img src="./img/finish-setup-public-access.png" width="500" />

หลังจากที่เซตเสร็จเรียบร้อยแล้ว ให้เราลองดึงข้อมูลที่เก็บอยู่ในรูปแบบ CSV จาก S3 โดยใช้คำสั่งประมาณนี้

In [5]:
df = spark.read.csv("s3a://kanouivirach-titanic/landing/titanic.csv", header=True)

In [6]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [7]:
df.createOrReplaceTempView("titanic")

In [ ]:
spark.sql("""
    select
        name
        , survived
        
    from
        titanic
""").show()

+--------------------+--------+
|                name|survived|
+--------------------+--------+
|Braund, Mr. Owen ...|       0|
|Cumings, Mrs. Joh...|       1|
|Heikkinen, Miss. ...|       1|
|Futrelle, Mrs. Ja...|       1|
|Allen, Mr. Willia...|       0|
|    Moran, Mr. James|       0|
|McCarthy, Mr. Tim...|       0|
|Palsson, Master. ...|       0|
|Johnson, Mrs. Osc...|       1|
|Nasser, Mrs. Nich...|       1|
|Sandstrom, Miss. ...|       1|
|Bonnell, Miss. El...|       1|
|Saundercock, Mr. ...|       0|
|Andersson, Mr. An...|       0|
|Vestrom, Miss. Hu...|       0|
|Hewlett, Mrs. (Ma...|       1|
|Rice, Master. Eugene|       0|
|Williams, Mr. Cha...|       1|
|Vander Planke, Mr...|       0|
|Masselmani, Mrs. ...|       1|
+--------------------+--------+
only showing top 20 rows



In [ ]:
table = spark.sql("""
    select
        name
        , survived
        
    from
        titanic
""")

In [ ]:
table.write.mode("overwrite").csv("s3a://kanouivirach-titanic/titanic.parquet")